In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Clone Tensorflow Models

In [0]:
!git clone https://github.com/tensorflow/models.git /content/drive/My\ Drive/colab/tensorflow-models
!git clone https://github.com/cocodataset/cocoapi.git /content/drive/My\ Drive/colab/cocoapi

Cloning into '/content/drive/My Drive/colab/cocoapi'...
remote: Enumerating objects: 953, done.
remote: Total 953 (delta 0), reused 0 (delta 0), pack-reused 953
Receiving objects: 100% (953/953), 11.70 MiB | 1.22 MiB/s, done.
Resolving deltas: 100% (565/565), done.


## Install Object Dection

In [0]:
os.chdir("/content/drive/My Drive/colab/cocoapi/PythonAPI")
!make
!cp -r pycocotools /content/drive/My\ Drive/colab/tensorflow-models/research/

In [0]:
os.chdir("/content/drive/My Drive/colab/tensorflow-models/research")
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
os.environ["PYTHONPATH"] = os.environ["PYTHONPATH"]+":/content/drive/My Drive/colab/tensorflow-models/research:/content/drive/My Drive/colab/tensorflow-models/research/slim"
!echo $PYTHONPATH

/env/python:/content/drive/My Drive/colab/tensorflow-models/research:/content/drive/My Drive/colab/tensorflow-models/research/slim


In [0]:
# Test installation
import os
os.chdir("/content/drive/My Drive/colab/tensorflow-models/research")
!python object_detection/builders/model_builder_test.py

## download model

In [0]:
os.chdir("/content/drive/My Drive/colab/tennis_detection/models/")
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -O ssdlite_mobilenet_v2_coco.tar.gz
!tar -zvxf ssdlite_mobilenet_v2_coco.tar.gz -C ssdlite_mobilenet_v2_coco
!rm ssdlite_mobilenet_v2_coco.tar.gz

## Training

In [0]:
!python /content/drive/My\ Drive/colab/tensorflow-models/research/object_detection/model_main.py  \
--pipeline_config_path=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/training/pipeline.config \
--model_dir=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/training  \
--num_train_steps=100000   \
--sample_1_of_n_eval_examples=1

## Export Graph

In [0]:
!python /content/drive/My\ Drive/colab/tensorflow-models/research/object_detection/export_inference_graph.py \
--input_type=image_tensor  \
--pipeline_config_path=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/training/pipeline.config  \
--trained_checkpoint_prefix=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/training/model.ckpt-100000  \
--output_directory=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/output

## Export TFlite Graph

In [0]:
!python /content/drive/My\ Drive/colab/tensorflow-models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/output/pipeline.config \
--trained_checkpoint_prefix=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/output/model.ckpt \
--output_directory=/content/drive/My\ Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/output/tflite \
--add_postprocessing_op=true

In [0]:
os.chdir("/content/drive/My Drive/colab/tennis_detection/models/ssdlite_mobilenet_v2_coco/output/")
!tflite_convert \
--graph_def_file=tflite/tflite_graph.pb \
--output_file=tflite/detect.tflite \
--output_format=TFLITE \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--mean_values=128 \
--std_dev_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops